In [ ]:
from sympy import *
from IPython.display import display, Math, Latex, clear_output, Image
import numpy as np
import cupy as cp
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from skimage.feature import peak_local_max
from scipy.ndimage import center_of_mass
from scipy.spatial import Delaunay
from collections import Counter
import os

In [ ]:
import sys
sys.path.append('D:/GitHubWSU/pfc-blocks/python/pfc-models/ln-vacancy')

import importlib

import PFC2D_VacancyMQ_CPU as PFC2D_Vacancy
importlib.reload(PFC2D_Vacancy)

In [ ]:
# import sys
# sys.path.append('D:/GitHubWSU/pfc-blocks/python/pfc-models/ln-vacancy')

# import importlib

# import PFC2D_Vacancy as PFC2D_Vacancy
# importlib.reload(PFC2D_Vacancy)

In [ ]:
_b = (0, 1, 2)
_q = (2, 3, 4)
_k2 = -1

sum([(_bi**2 + (_qi**2 + _k2)**2) for _bi, _qi in zip(_b, _q)])


In [ ]:
beta_val = 0.02 # 1.0
epsilon_val = -0.15 #-2.82 -0.1
g_val = 0 #-1
phi_s = -0.2 #0.54  -0.1 #-0.18 #0.5 #
phi_l = -0.3 #0.32676978996474756
q_min = 1 #0.9718466499519173 #1.0
strain = 0 #0.20
q_eff = q_min * (1 + strain)
noiseChangeRate = 8

sim = PFC2D_Vacancy.PFC2D_Vacancy()
sim.minLog = -100.0
sim.minPhi = np.exp(sim.minLog)
sim.phiMax = 50
sim.parms.epsilon = epsilon_val
sim.parms.a = 0
sim.parms.beta = beta_val
sim.parms.b = (0, -0.2, 0.0)
sim.parms.q = (1, 3**0.5, 2)
sim.parms.g = g_val
sim.parms.v0 = 1.0
sim.parms.phi0 = phi_s

sim.parms.Hng = 0
sim.parms.Hln = 0
sim.parms.N = 288
sim.parms.Nx = 288
sim.parms.Ny = 288
sim.parms.PPU = 30
sim.parms.scaleFactor = 1/q_eff
sim.parms.eta = 1e-0 #0.0025
sim.parms.dt = 1e-2
sim.parms.seed = 34125
sim.parms.NoiseDynamicsFlag = False
sim.parms.NoiseTimeSmoothFlag = True
sim.parms.NoiseTimeSmoothingFrames = 0
sim.parms.Noise_CutoffOmega = 0.04

sim.parms.phi0 = phi_s
sim.InitParms()
# sim.SetGeometry(288, 288, 15, scalefactor = 1/q_eff)
sim.parms.Noise_CutoffK = 2*np.pi/(sim.dx*sim.nx) * sim.mx

# sim.InitFieldCrystal(A = np.sign(sim.parms.phi0) * sim.parms.phi0/3.1, noisy=False, scalefactor = 1/q_eff)
sim.InitFieldFlat(noisy=True)
# sim.phi = cp.sin(sim.x)
# sim.phi[144-20:144+20,144:144+40] = phi_l
# sim.phi[-64:,:] = phi_l
# sim.phi[:,144-72:144+72] = sim.phi[144-72:144+72,:].T
sim.t = 0


# Total steps and checkpoints
total_steps = int(1e3)
energy_calcs = 10

sim.CalcEnergyDensity()

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

phi_0 = sim.phi#.get()

In [ ]:
# phi_0 = sim.phi#.get()

In [ ]:
# sim.phi.mean()

In [ ]:
# sim.parms.eta = 0.5
# sim.parms.NoiseDynamicsFlag = True
# sim.parms.NoiseTimeSmoothFlag = True
# sim.parms.NoiseTimeSmoothingFrames = 10
# sim.parms.NoiseChangeRate = 20
# sim.InitializeTimeSmoothNoise()


In [ ]:
sim.SetDT(1e-0)
total_steps = int(1e4)

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
sim.SetDT(1e-1)
total_steps = int(1e4)

# Run simulation
for i in range(total_steps):

  try:
    # Show energy, progress
    if i % int(total_steps/energy_calcs) == int(total_steps/energy_calcs)-1:
      sim.CalcEnergyDensity()
      print(f'f: {sim.f:.10f}, t: {sim.t:.2f}, min: {sim.phi.min()}, max: {sim.phi.max()}, mean: {sim.phi.mean()}')
      sim.SetDT(sim.parms.dt * 0.1)

    # Step simulation
    sim.TimeStepCross()
  except Exception as e:
    print(e)
    break

In [ ]:
fig = go.Figure(data=go.Heatmap(x=np.arange(sim.nx)*sim.dx,
                                y=np.arange(sim.ny)*sim.dy,
                                z=sim.phi.get(), colorscale='gray', showscale=False))

# decide pixels per cell (1 means 1 figure pixel per heatmap cell)
px_per_cell = 2

# exact figure pixel size to match heatmap grid
fig_width = int(sim.nx * px_per_cell)
fig_height = int(sim.ny * px_per_cell)

fig.update_layout(
    width=fig_width,
    height=fig_height,
    autosize=False,
    margin=dict(l=0, r=0, t=0, b=0),
    plot_bgcolor='white',
    # place colorbar inside plot or hide it to avoid extra space
    # if you want a colorbar keep it small and place it on top of the image
    # e.g. fig.data[0].colorbar.update({'len':1, 'thickness':8, 'x':0.995})
)

fig.update_xaxes(
    showgrid=False, showticklabels=False,
    # force axis to use full horizontal domain
    domain=[0.0, 1.0],
    range=[0, sim.nx * sim.dx]   # match the heatmap coordinates
)

fig.update_yaxes(
    showgrid=False, showticklabels=False,
    domain=[0.0, 1.0],
    range=[0, sim.ny * sim.dy],  # match the heatmap coordinates
    scaleanchor='x',             # lock aspect ratio so a unit in x equals unit in y
    scaleratio=1
)

# optionally hide the colorbar to remove its margin impact
if hasattr(fig.data[0], 'colorbar'):
    fig.data[0].colorbar.update({'thickness': 8, 'len': 1, 'x': 0.995})  # place thin bar on right
    # or remove it: fig.data[0].colorbar = dict() or fig.data[0].showscale = False

fig.show()

In [ ]:
fig.write_image("./out/prospectus/fig-honeycomb-multiq1.png")